In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os

print(os.listdir("/content/drive/MyDrive/fine_tuning_volgare"))


['model.safetensors', 'generation_config.json', 'config.json', 'training_args.bin', 'tokenizer_config.json', 'special_tokens_map.json', 'sentencepiece.bpe.model', 'tokenizer.json']


In [4]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments


model_path = "/content/drive/MyDrive/fine_tuning_volgare"  # o "/content/drive/MyDrive/fine_tuning_volgare"

tokenizer = MBart50TokenizerFast.from_pretrained(model_path)
model = MBartForConditionalGeneration.from_pretrained(model_path)


In [5]:
import pandas as pd

dataset_id = "1KfvUPTtutsF63NxNXLtoFJQRASm1WwP3" #dataset_cleaned
url = f"https://drive.google.com/uc?id={dataset_id}"

df = pd.read_csv(url)
head = pd.read_csv(url, nrows=5)


In [6]:
#Trial with the first 5 rows in head
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for frase in head['Sentence']:
    # Aggiungi il prefisso usato nel training
    input_text = f"translate: {frase}"

    # Tokenizza con il prefisso incluso
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

    # Genera output
    output_ids = model.generate(
        input_ids,
        max_length=300,             # aumenta se il testo è lungo
        num_beams=4,
        early_stopping=True
    )

    # Decodifica la traduzione
    traduzione = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    print(f"Volgare: {frase}")
    print(f"Moderno: {traduzione}\n")



Volgare: quella guerra ben fatta l' opera perché etc. Et dall' altra parte Aiaces era uno cavaliere franco e prode all' arme, di gran guisa, ma non era pieno di grande senno
Moderno: E dall'altra parte Aiaces fu un cavallere franco e proso all'arma, con una grande volontà, ma non era pieno di grande senno.

Volgare: crudele, e di tutte le colpe pigli vendetta, come dice la legge, ed a neuno cavaliere perdoni che pecchi.
Moderno: È crudele, e di tutte le colpe si fa vendetta, come dice la legge, e perdonò a un giovane cavaliere che si fa male.

Volgare: Non d' altra forza d' animo fue ornato Ponzio Aufidiano, romano cavaliere.
Moderno: Non c'era altra forza d' animo che fu demostrata al Ponzio Aufidiano, un cavallere romano.

Volgare: Se questo piace a tutti e se 'l tempo hae bisogno d'avere Pompeio per cavaliere e non per compagno, non riterrò più i fati.
Moderno: Se questo piace a tutti e se il tempo ci vuole Pompei per il cavallo e non per il compagno, non riterrò più i fati.

Volgar

In [7]:
translations = []
for frase in df['Sentence']:
    # Same prefix as training
    input_text = f"translate: {frase}"

    # #Tokenizing the Sentence
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

    # Output
    output_ids = model.generate(
        input_ids,
        max_length=512,
        num_beams=4,
        early_stopping=True
    )

    #Decode the translation
    traduzione = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    translations.append(traduzione)


In [8]:
df['Translation'] = translations

In [9]:
df

,Author,Date,Region,Sentence,Translation
0,Brunetto Latini,1260-61,fior.,quella guerra ben fatta l' opera perché etc. E...,E dall'altra parte Aiaces fu un cavallere fran...
1,Bono Giamboni,1292,fior.,"crudele, e di tutte le colpe pigli vendetta, c...","È crudele, e di tutte le colpe si fa vendetta,..."
2,Valerio Massimo (red. V1,1336,fior.,Non d' altra forza d' animo fue ornato Ponzio ...,Non c'era altra forza d' animo che fu demostra...
3,Lucano volg. (ed. Marinoni),1330/40,prat.,Se questo piace a tutti e se 'l tempo hae biso...,Se questo piace a tutti e se il tempo ci vuole...
4,Brunetto Latini,1260-61,fior.,Officio di questa arte pare che sia dicere app...,L'intento di questa arte sembra essere solo pe...
...,...,...,...,...,...
92,Grazia di Meo,1343,tosc.,"Alchuno è riccho e gentile, ma lamentasi che v...","Alchuno è ricco e gentile, ma lamentava di ave..."
93,Zanobi da Strada,1361,tosc.,"Pietro, essendogli mostrato in figura il popul...","Pietro, visto che gli era stato mostrato in fi..."
94,Declamazioni di Seneca,1392,tosc.,pregollo che lo liberasse di quella obbligazio...,"Il gentile uomo si addessì, e si liberò, e fec..."
95,Giovanni da San Miniato,1415,tosc.,L'oro verrà dall'Aquilone. Che figuriamo noi p...,"L'oro verrà dall'Aquilone, a parte noi per l'A..."


In [10]:
# Salva in formato JSONL
df.to_json('giuminia-hw2_transl-finetuned-model.jsonl', orient='records', lines=True, force_ascii=False)

In [14]:
import shutil

source_path = '/content/giuminia-hw2_transl-finetuned-model.jsonl'
destination_path = '/content/drive/MyDrive/model_jsonl/giuminia-hw2_transl-finetuned-model.jsonl'


shutil.copy(source_path, destination_path)


'/content/drive/MyDrive/model_jsonl/giuminia-hw2_transl-finetuned-model.jsonl'